In [ ]:
p_tenant_id = "<TENANT_ID>"
p_azure_key_vault = "<AZURE_KEY_VAULT>"
p_workspace_id="<WORKSPACE_ID>"
p_lakehouse_id="<LAKEHOUSE_ID>"
p_table_name="<LAKEHOUSE_TABLE_NAME>"
# Key Vault secret keys for Service Principal
p_secret_client_id = "<SECRET_CLIENT_ID>"
p_secret_client_secret = "<SECRET_CLIENT_SECRET>"

In [ ]:
storage_account_name = "onelake"
workspace_id=p_workspace_id
lakehouse_id=p_lakehouse_id
table_name=p_table_name
endpoint=f"{storage_account_name}.dfs.fabric.microsoft.com"
path = f"abfss://{workspace_id}@{endpoint}/{lakehouse_id}/Tables/{table_name}/"

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

kv_uri = f"https://{p_azure_key_vault}.vault.azure.net/"

credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=kv_uri, credential=credential)

client_id = secret_client.get_secret(p_secret_client_id).value
client_secret = secret_client.get_secret(p_secret_client_secret).value

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("fabric_conn") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "org.apache.hadoop:hadoop-azure-datalake:3.3.1,"
            "io.delta:delta-spark_2.12:3.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

spark.conf.set(f"fs.azure.account.auth.type.{endpoint}", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{endpoint}",  "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{endpoint}", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{endpoint}",client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{endpoint}", f"https://login.microsoftonline.com/{p_tenant_id}/oauth2/token")

print(f"ABFS Path: {path}")
df = spark.read.format("delta").load(path)
df.show()